*Note: you should use the default (non-GPU / non-TPU) version of Colab runtime for this tutorial. There might be some Linux differences in the non-free environment that can affect stuff described below, plus Colab will nag you about GPU staying idle anyway.*

As per the usual Colab routine, start with clicking "Copy to Drive" button above, then execute the following cell to get the sources and navigate to the relevant folder:

In [ ]:
!git clone https://github.com/nsndp/dbus-example
%cd dbus-example

Let's also give executable permissions to every shell script, as well as to prebuilt C binaries:

In [ ]:
!find . -type f -iname "*.sh" -exec chmod -v +x {} \;
!chmod -v +x gserver gclient

# Installing prerequisites

You can only run the cells for the languages you're interested in; the rest should still work as long as you stick to those languages. You can even skip this section altogether if you're fine with only C examples using prebuilt binaries.


Also note that, since Colab virtual environments keep being updated, some of the installation particulars below might change - hopefully not by much, but feel free to open an issue back at GitHub if something breaks.

For C's [GLib](https://github.com/GNOME/glib), everything is straightforward:

In [ ]:
!apt-get install libglib2.0-dev

For Python's [dasbus](https://github.com/rhinstaller/dasbus), PyGObject should be preinstalled, but it seems to have problems when multiple Python versions are present (as is the case in Colab), and we circumvent it by switching back to 3.8:

In [ ]:
!update-alternatives --set python3 /usr/bin/python3.8
!apt install python3-pip
!pip install dasbus

For Java's [dbus-java](https://github.com/hypfvieh/dbus-java), Open JDK 11 should be preinstalled, so we only need to add Maven as a building tool:

In [ ]:
!apt-get install maven

Alternatively,  we can get a newer JDK version and then do a manual installation of fresh Maven as instructed on their [website](https://maven.apache.org/install.html) (uncomment the lines below for this route).

In [ ]:
#!apt-get install openjdk-17-jdk
#!wget https://dlcdn.apache.org/maven/maven-3/3.9.1/binaries/apache-maven-3.9.1-bin.tar.gz -P /content/
#!tar xzvf /content/apache-maven-3.9.1-bin.tar.gz -C /content/
#!rm -f /content/apache-maven-3.9.1-bin.tar.gz
#OLD_ENV_PATH = %env PATH
#%env PATH=$OLD_ENV_PATH:/content/apache-maven-3.9.1/bin

For PHP's [pecl-dbus](https://github.com/derickr/pecl-dbus), first we get PHP and its command-line tool:

In [ ]:
!apt-get install php-dev
!apt-get install php-cli

And then we build the extension from source as per [instructions](https://github.com/derickr/pecl-dbus/blob/master/README.rst#building) (the source itself can then be safely removed):

In [ ]:
!apt-get install libdbus-1-dev
!cd /content && git clone https://github.com/derickr/pecl-dbus
!cd /content/pecl-dbus && phpize && ./configure && make && make install
!rm -rf /content/pecl-dbus

Alternatively, we have a prebuilt version of the same extension that can copied to the relevant PHP directory (uncomment the line below for this route). Note that, however, it would only work for the same PHP version it was built with (PHP 7.4).

In [ ]:
#!cp php-ext/dbus.so $(php -i | grep extension_dir | cut -d " " -f 5)

# Building

For C, prebuilt binaries *gserver* and *gclient* are included in the project's root. You can, however, rebuild them by running:

In [ ]:
!make -C gdbus_server
!make -C gdbus_client

For Java, run the following to do builds for server+client inside their respective directories and then copy .jar results to the project's root:

In [ ]:
!cd java-server && mvn package && cd ..
!cd java-client && mvn package && cd ..
!cp java-server/target/java-server-1.0-SNAPSHOT.jar java-server.jar
!cp java-client/target/java-client-1.0-SNAPSHOT.jar java-client.jar

For Python and PHP, no building is necessary, them being interpreted languages.

Alternatively, there's a script s_build.sh included that can do the same thing (uncomment the line and choose an appropriate flag for this route):

In [ ]:
#!./s_build.sh [all|c|java]

In can also clean up afterwards if needed:

In [ ]:
#!./s_build.sh clean

# Starting/stopping D-Bus

Since free Colab runs inside a container, we'll need to start D-Bus service manually and set the neccessary environment variable like this:

In [ ]:
!export DBUS_SESSION_BUS_ADDRESS=$(dbus-daemon --session --fork --print-address)

*dbus-daemon* utility is part of *dbus* package that should be preinstalled. There is also *dbus-launch* utility from *dbus-x11* package that can do the same thing with a shorter command:

In [ ]:
!apt-get install dbus-x11
!export $(dbus-launch)

However, Colab/Jupyter treats every ! command as its own terminal session, which means that any env var set with *export* is lost beyond it, so we'll need to launch these commands in conjunction with running server+client using "**&&**" or "**;**". For now, let's list all running processes to confirm that the two dbus-daemons from above are up, then stop them both by name (-e option for pkill means to print the results).

In [ ]:
!ps -ef
!pkill -e dbus-daemon

The above applies when using D-Bus session bus, which is more common and hence is the default mode for our servers/clients. Starting the system bus is a bit more distro-specific, but on Ubuntu/Debian systems can usually be achieved with a built-in init script like this:

In [ ]:
!/etc/init.d/dbus start

In [ ]:
!/etc/init.d/dbus stop

# Running

The following command will start the session bus, launch C server in background, give it a bit of time to initialize, launch C client, wait for it to finish and then stop the bus. You should see the expected test output (4 workers with 4 instances each) as described in README.

In [ ]:
!export $(dbus-launch) && (./gserver &) && sleep 1 && (./gclient) && pkill -e dbus-daemon

The analogous commands for other languages' implementations are as follows:

In [ ]:
!export $(dbus-launch) && \
(python3 dasbus_server.py &) && sleep 1 && \
(python3 dasbus_client.py) && pkill -e dbus-daemon

In [ ]:
!export $(dbus-launch) && \
(java -cp java-server.jar com.exmpl.server.Server &) && sleep 5 && \
(java -cp java-client.jar com.exmpl.client.Client) && pkill -e dbus-daemon
# dbus-java takes a lot of time to initialize the server for some reason, hence sleep 5 instead of 1

In [ ]:
!export $(dbus-launch) && \
(php -d extension=dbus.so -f pecldbus_server.php &) && sleep 1 && \
(php -d extension=dbus.so -f pecldbus_client.php) && pkill -e dbus-daemon

And doing the same with C but on system bus (with copying a prepared config file to enable necessary permissions first) would be:

In [ ]:
!cp com.exmpl.v0001.conf /usr/share/dbus-1/system.d
!/etc/init.d/dbus start && \
(./gserver --system &) && sleep 1 && \
(./gclient --system) && \
/etc/init.d/dbus stop

There's also s_run.sh script included that includes all of the above. Feel free to run it using any combination of params and confirm that results stay the same:

In [ ]:
#!./s_run.sh               # run C server and C client (default)
#!./s_run.sh c c           # same as the above
#!./s_run.sh py py         # run Python server and Python client
#!./s_run.sh java php      # run Java server and PHP client
#!./s_run.sh php java      # run PHP server and Java client
#!./s_run.sh c c --system  # run C server and C client on system bus

Finally, there's s_run_multi.sh script that can start multiple clients simultaneously like this:

In [ ]:
!./s_run_multi.sh c c+c+py+php